In [ ]:
import os

os.mkdir('../comp_submissions/')

### Data imports

In [ ]:
import pandas as pd

url = "https://driv######################link"
path = "https://drive.google.com/uc?export=download&id="+url.split('/')[-2]
data = pd.read_csv(path, index_col="Id")

test_url = "https://drive.##############################_link"
test_path = "https://drive.google.com/uc?export=download&id="+test_url.split('/')[-2]
test_data = pd.read_csv(test_path, index_col="Id")

data

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,8450,65.0,856,3,0,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,9600,80.0,1262,3,1,0,2,298,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,11250,68.0,920,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
4,9550,60.0,756,3,1,0,3,0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
5,14260,84.0,1145,4,1,0,3,192,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,7917,62.0,953,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1457,13175,85.0,1542,3,2,0,2,349,0,0,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1458,9042,66.0,1152,4,2,0,1,0,0,1,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal


### Intuition model

In [ ]:
def dummy_function(df):
    df['Expensive'] = df.apply(lambda x: 1 if x['Fireplaces']>=1 else 0, axis=1)
    return df

dummy_function(data)['Expensive']

Id
1       0
2       1
3       1
4       1
5       1
       ..
1456    1
1457    1
1458    1
1459    0
1460    0
Name: Expensive, Length: 1460, dtype: int64

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    FunctionTransformer(dummy_function).set_output(transform="pandas")
)

pipe.fit(data)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function dummy_function at 0x000001F2AC52A340>))])

In [ ]:
prediction_data = pipe.transform(data)
prediction_data['Expensive']

Id
1       0
2       1
3       1
4       1
5       1
       ..
1456    1
1457    1
1458    1
1459    0
1460    0
Name: Expensive, Length: 1460, dtype: int64

To submit to the competition, predictions must be made on `test_data`.

In [ ]:
submission = pipe.transform(test_data)[["Expensive"]]
submission

submission.to_csv("../comp_submissions/classification_dummy.csv")

### KNN

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn import set_config
set_config(transform_output="pandas")

In [ ]:
y = data['Expensive'].copy()
X = data.drop('Expensive', axis=1).copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_num = X_train.select_dtypes(include='number')
X_cat = X_train.select_dtypes(exclude='number')

In [ ]:
num_pipe = make_pipeline(SimpleImputer(strategy='mean'))
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='N_A'),
                        OneHotEncoder(handle_unknown='infrequent_if_exist',
                                      sparse_output=False))
preprocessor = make_column_transformer((num_pipe, X_num.columns),
                                       (cat_pipe, X_cat.columns))

knn_pipe = make_pipeline(preprocessor, KNeighborsClassifier())

In [ ]:
knn_pipe.fit(X_train, y_train)
accuracy_score(y_true=y_train,
               y_pred=knn_pipe.predict(X_train))

0.797945205479452

In [ ]:
knn_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('pipeline-1',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer())]),
                                    Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
          'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
          'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
          'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF...
          'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
          'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
          'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2',
          'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
          'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
          'MiscFeature', 'Sa

In [ ]:
knn_pipe.set_params(kneighborsclassifier__n_neighbors=15, kneighborsclassifier__weights='distance')
knn_pipe.fit(X_train, y_train)
accuracy_score(y_true=y_train,
               y_pred=knn_pipe.predict(X_train))

1.0

In [ ]:
knn_pipe.score(X_test, y_test)

0.7157534246575342

In [ ]:
knn_pipe.predict(X_test)

array([1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0], dtype=int64)

The output of `.predict()` is an array. This will have to be joined with `Id`s to create a suitable submission.

In [ ]:
prediction = X_test.copy()
prediction['Expensive'] = knn_pipe.predict(X_test)
prediction['Expensive']

Id
148     1
677     1
1305    0
1373    1
1428    1
       ..
1013    1
1060    1
891     0
629     1
353     0
Name: Expensive, Length: 292, dtype: int64

To maximize the use of our training data, we can `fit` one last time on the _full_ (unsplit) training data.

True prediction is then done on `test_data`.

In [ ]:
knn_pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF...
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2',
       'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=15, weights='distance'))])

In [ ]:
knn_pred = test_data.copy()
knn_pred['Expensive'] = knn_pipe.predict(test_data)
knn_pred['Expensive'].to_csv('../comp_submissions/classification_knn_untrimmed.csv')